## Data-driven optimization and decision making - Assignment 2
Juha Reinikainen

In [1]:
from scipy.stats import qmc
from desdeo_problem import variable_builder, \
    ScalarConstraint, MOProblem, ScalarObjective
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor, kernels
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
# from sklearn import preprocessing
import warnings
#hide gridsearchcv convergence warnings
warnings.simplefilter("ignore")
seed = 1
n_folds = 5


In [2]:
# construct the problem
# https://www.mathworks.com/help/gads/multiobjective-optimization-welded-beam.html
def F1(x):
    """
    minimize the fabrication cost of the beam
    """
    return 1.10471 * x[:, 0]**2 * x[:, 1] + 0.04811 * x[:, 2] * x[:, 3] * (14.0 + x[:, 1])


P = 6000.0  # max supported load of the beam
L = 14.0  # length of part


def F2(x):
    """
    Minimize the deflection of the beam
    """
    C = (4*(14**3))/(30*10**6)
    return (P / (x[:, 3] * x[:, 2]**3)) * C


objectives = [
    ScalarObjective("cost", F1, maximize=False),
    ScalarObjective("deflection", F2, maximize=False)
]

"""
Welded beam design problem
x[0] the thickness of the welds
x[1] the length of the welds
x[2] the height of the beam
x[3] the width of the beam
"""
var_names = ["thickness", "length", "height", "width"]
lb = [0.125, 0.1, 0.1, 0.125]
ub = [5.0, 10.0, 10.0, 5.0]
initial_values = [0.125, 0.1, 0.1, 0.125]
variables = variable_builder(var_names, initial_values, lb, ub)


def shear_stress(x, y):
    r1 = 1 / np.sqrt(2 * x[:, 0] * x[:, 1])
    R = np.sqrt(x[:, 1]**2 + (x[:, 0] + x[:, 2])**2)
    r2 = ((L + x[:, 1]/2) * R) / np.sqrt(2 * x[:, 0] *
                                         x[:, 2] * ((x[:, 1]**2)/3 + (x[:, 0] + x[:, 2])**2))
    r = P * np.sqrt(r1**2 + r2**2 + (2*r1*r2*x[:, 1])/R)
    return 13600 - r


constraints = [
    ScalarConstraint("thickness", 4, 2, lambda x, y: x[:, 3] - x[:, 0]),
    ScalarConstraint("shear stress", 4, 2, shear_stress),
    ScalarConstraint("normal stress", 4, 2, lambda x,
                     y: 30000 - P * (6 * L)/(x[:, 3]*x[:, 2]**2)),
    ScalarConstraint("buckling load", 4, 2, lambda x, y: (
        64746.022 * (1-0.0282346*x[:, 2])*x[:, 2]*x[:, 3]**3) - 6000),
]

prob = MOProblem(objectives=objectives, variables=variables,
                 constraints=constraints)


In [3]:
"""
Generate data
"""

# Generate samples and scale them based on the box constraints
# and minmax scaling because the objective function values have large range
lhs = qmc.LatinHypercube(4, seed=seed)
X = lhs.random(100)
X = qmc.scale(X, lb, ub)
res = prob.evaluate(X)
y = res.objectives
# y = preprocessing.MinMaxScaler().fit_transform(y)
y1 = y[:, 0]
y2 = y[:, 1]

# test samples
X_test = lhs.random(50)
X_test = qmc.scale(X_test, lb, ub)
res_test = prob.evaluate(X_test)
y_test = res_test.objectives
# y_test = preprocessing.MinMaxScaler().fit_transform(y_test)
y1_test = y_test[:, 0]
y2_test = y_test[:, 1]

print(X.min(axis=0), "\n", X.max(axis=0))
print(y.min(axis=0), y.max(axis=0))


[0.15176185 0.15909181 0.13072661 0.13322651] 
 [4.96363502 9.90438302 9.97704191 4.99043053]
[6.5883435e-01 5.6626839e-04] [219.66098571 503.10180463]


In [4]:
"""
Cross validate random forest
objective 2 only depends on the two last variables so only
use those for fitting for all y2 models
"""

randomForestRegressor_y1 = RandomForestRegressor(random_state=seed)
randomForestRegressor_y2 = RandomForestRegressor(random_state=seed)
randomForestRegressor_y1.fit(X,y1)
randomForestRegressor_y2.fit(X[:,2:],y2)

scoring = ["r2", "neg_root_mean_squared_error"]

scores_y1 = cross_validate(randomForestRegressor_y1,
                           X, y1, scoring=scoring, cv=n_folds)
scores_y2 = cross_validate(randomForestRegressor_y2,
                           X[:,2:], y2, scoring=scoring, cv=n_folds)
print("objective 1")
print("r2", scores_y1["test_r2"])
print("mean", scores_y1["test_r2"].mean())
print("var", scores_y1["test_r2"].var())
print("rmse", -scores_y1["test_neg_root_mean_squared_error"])
print("mean", (-scores_y1["test_neg_root_mean_squared_error"]).mean())
print("var", (-scores_y1["test_neg_root_mean_squared_error"]).var())

print("objective 2")
print("r2", scores_y2["test_r2"]) 
print("mean", scores_y2["test_r2"].mean()) 
print("var", scores_y2["test_r2"].var())
print("rmse", -scores_y2["test_neg_root_mean_squared_error"])
print("mean", (-scores_y2["test_neg_root_mean_squared_error"]).mean())
print("var", (-scores_y2["test_neg_root_mean_squared_error"]).var())


objective 1
r2 [0.88131703 0.80597788 0.92493396 0.91108308 0.91605465]
mean 0.887873318826464
var 0.0018912439550861262
rmse [16.85917701 18.80036625 13.56853671 13.09881632 14.57951492]
mean 15.381282241657715
var 4.6025672500174695
objective 2
r2 [  0.65998944   0.05645192  -1.51788178 -19.06785278   0.87095429]
mean -3.7996677835413406
var 58.979386573927876
rmse [  1.01990603 106.50289908   9.19476714  30.24456323   0.13067523]
mean 29.41856214413365
var 1603.1886305197927


In [5]:
"""
tune hyperparameters and crossvalidate Kriging
"""

parameters = {
    "alpha": np.logspace(-10, 1, 5),
    "kernel": [
        kernels.RBF(),
        kernels.ExpSineSquared(),
        kernels.DotProduct(),
        kernels.RationalQuadratic(),
    ],
    # "normalize_y": [False, True]
}
print(parameters["alpha"])

gaussianProcessRegressor_y1 = \
    GridSearchCV(GaussianProcessRegressor(
                                          n_restarts_optimizer=5, 
                                          random_state=seed), parameters, cv=n_folds)
gaussianProcessRegressor_y2 = \
    GridSearchCV(GaussianProcessRegressor(
                                          n_restarts_optimizer=5, 
                                          random_state=seed), parameters, cv=n_folds)
# fit best parameters
gaussianProcessRegressor_y1.fit(X, y1)
gaussianProcessRegressor_y1 = gaussianProcessRegressor_y1.best_estimator_
gaussianProcessRegressor_y2.fit(X[:,2:], y2)
gaussianProcessRegressor_y2 = gaussianProcessRegressor_y2.best_estimator_
print(gaussianProcessRegressor_y1)
print(gaussianProcessRegressor_y2)

# 5-folds
scores_y1 = cross_validate(gaussianProcessRegressor_y1,
                           X, y1, scoring=scoring, cv=n_folds)
scores_y2 = cross_validate(gaussianProcessRegressor_y2,
                           X[:,2:], y2, scoring=scoring, cv=n_folds)

print("objective 1")
print("r2", scores_y1["test_r2"])
print("mean", scores_y1["test_r2"].mean())
print("var", scores_y1["test_r2"].var())
print("rmse", -scores_y1["test_neg_root_mean_squared_error"])
print("mean", (-scores_y1["test_neg_root_mean_squared_error"]).mean())
print("var", (-scores_y1["test_neg_root_mean_squared_error"]).var())

print("objective 2")
print("r2", scores_y2["test_r2"]) 
print("mean", scores_y2["test_r2"].mean()) 
print("var", scores_y2["test_r2"].var())
print("rmse", -scores_y2["test_neg_root_mean_squared_error"])
print("mean", (-scores_y2["test_neg_root_mean_squared_error"]).mean())
print("var", (-scores_y2["test_neg_root_mean_squared_error"]).var())


[1.00000000e-10 5.62341325e-08 3.16227766e-05 1.77827941e-02
 1.00000000e+01]
GaussianProcessRegressor(alpha=3.1622776601683795e-05,
                         kernel=ExpSineSquared(length_scale=1, periodicity=1),
                         n_restarts_optimizer=5, random_state=1)
GaussianProcessRegressor(alpha=10.0,
                         kernel=ExpSineSquared(length_scale=1, periodicity=1),
                         n_restarts_optimizer=5, random_state=1)
objective 1
r2 [0.99681554 0.99310156 0.99642601 0.99109819 0.9827452 ]
mean 0.9920372994513402
var 2.608998026379606e-05
rmse [2.76159753 3.54499953 2.96065663 4.14456525 6.60997263]
mean 4.004358312015205
var 1.930732399940655
objective 2
r2 [-0.07988484 -0.03819489  0.86959511 -0.16648495  0.06763787]
mean 0.1305336594683512
var 0.14222660516253888
rmse [  1.81761828 111.7168946    2.0925219    7.29182831   0.35124795]
mean 24.654022207251753
var 1900.5074006571908


In [6]:
"""
Compute metrics for both models on test data
"""
y1_pred_forest = randomForestRegressor_y1.predict(X_test)
y2_pred_forest = randomForestRegressor_y2.predict(X_test[:,2:])
print("random forest test")
print("r2 objective 1", r2_score(y1_test, y1_pred_forest))
print("rmse objective 1", mean_squared_error(
    y1_test, y1_pred_forest, squared=False))
print("r2 objective 2", r2_score(y2_test, y2_pred_forest))
print("rmse objective 2", mean_squared_error(
    y2_test, y2_pred_forest, squared=False))

y1_pred_kriging = gaussianProcessRegressor_y1.predict(X_test)
y2_pred_kriging = gaussianProcessRegressor_y2.predict(X_test[:,2:])
print("kriging test")
print("r2 objective 1", r2_score(y1_test, y1_pred_kriging))
print("rmse objective 1", mean_squared_error(
    y1_test, y1_pred_kriging, squared=False))
print("r2 objective 2", r2_score(y2_test, y2_pred_kriging))
print("rmse objective 2", mean_squared_error(
    y2_test, y2_pred_kriging, squared=False))


random forest test
r2 objective 1 0.9451871125660081
rmse objective 1 11.58173974486011
r2 objective 2 -0.0390106962515091
rmse objective 2 23.882504146127694
kriging test
r2 objective 1 0.9977810759053293
rmse objective 1 2.3302562657945987
r2 objective 2 0.005122087168357492
rmse objective 2 23.3697866280749


For k-fold CV the cell value contains mean and variance computed from invidual values of r-squared and rmse on each fold.

|  | | Objective 1 | | Objective 2 | |
| ---- | ---- | -- | -- | --       | -- |
|  | | Kriging | Random forest | Kriging | Random forest |
| k-fold CV | R-squared | 0.99 (2.6-05) | 0.89 (0.002) | 0.13 (0.142)| -3.80 (58.98) |
|  | RMSE |  4.0 (1.93) | 15.4 (4.60) | 24.7 (1900.51) |29.4 (1603.19) |
| Test | R-squared | 1.00 | 0.95 | 0.005 | -0.039 |
|  | RMSE | 2.3 | 11.6 | 23.4 | 23.9 |

From the r-squared and rmse scores on training set, Kriging performed significantly better reaching almost r-squared score of 1 and it has smaller rmse value so it should be used as surrogate for objective 1. Both models performed poorly on objective 2 but once again Kriging was a bit better. Random forest could probably perform better with some hyperparameter tuning.

On the test set the models behaved similarly to the training set.  